## Final Project Submission

Please fill out:
* Student name: Eduardo Calzadilla
* Student pace: self paced 
* Scheduled project review date/time: 
* Instructor name: Jeff Herman
* Blog post URL:


# Movie genre and title recommendations

#### Datasets: 
    1. IMDB
    2. Rotten Tomatoes
    3. TheMovieDB.org
    4. Box Office Mojo





## Contents

**A. [Exploratory Data Analysis](#EDA)** <br>
  1. [Initial exploration and cleanup](#Iec)<br>
  2. [Exploratory visualizations](#Ev)<br>
  3. [Transformations and joins](#Tj)<br>
  
**B.**
 

## Introduction & Summary

Throughout this notebook we will be exploring three datasets as highlighted above, in order to spot trend that could be indicative of higher succes capacity and return on investment (ROI). 

We will be focusing on the following four null hypotheses:
  1. Movies with higher ratings do not perform significantly better in terms of gross income and ROI.
  2. All genres perform equally in terms of ratings, gross income and ROI.
  3. There are no words that are indicative of higher performance in terms of ratings and ROI.
  4. There has been no change in terms of competition (number of movies coming out) 4 year prior to 2015 and 4 year after 2015. 

We have found that there are significant similarities between median ratings and median performance. etc...

In [96]:
# Importing necessary packages
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import glob
import zipfile
%matplotlib inline

In [8]:
# Passing zip doc filenames to a list for storing DataFrames. 
zippedData = glob.glob("zippedData/*.gz")

In [206]:
# Verifyig that all the docs in zippedData were read correctly
zippedData

['zippedData/imdb.title.crew.csv.gz',
 'zippedData/tmdb.movies.csv.gz',
 'zippedData/imdb.title.akas.csv.gz',
 'zippedData/imdb.title.ratings.csv.gz',
 'zippedData/imdb.name.basics.csv.gz',
 'zippedData/rt.reviews.tsv.gz',
 'zippedData/imdb.title.basics.csv.gz',
 'zippedData/rt.movie_info.tsv.gz',
 'zippedData/tn.movie_budgets.csv.gz',
 'zippedData/bom.movie_gross.csv.gz',
 'zippedData/imdb.title.principals.csv.gz']

In [190]:
# Ceate list of variables for storing DataFrames.
df_name=[]
for filename in zippedData:
    df_name.append("df_" + filename.split("/",1)[1].split(".",3)[0] 
                + "_" + filename.split("/",1)[1].split(".",3)[1] 
                + "_" + filename.split("/",1)[1].split(".",3)[2])

In [199]:
df_dic={}
i = 0
for filename in zippedData:
    if "csv" in filename:
        df_dic[df_name[i]] = pd.read_csv(filename,
                             compression='gzip')
    else: 
        df_dic[df_name[i]] = pd.read_csv(filename
                                   , sep='\t'
                                   , encoding = 'unicode_escape')
    i+=1
    

In [201]:
df_imdb_title_crew = df_dic["df_imdb_title_crew"]
df_imdb_title_akas = df_dic["df_imdb_title_akas"]
df_imdb_title_ratings = df_dic["df_imdb_title_ratings"]
df_imdb_name_basics = df_dic["df_imdb_name_basics"]
df_imdb_title_basics = df_dic["df_imdb_title_basics"]
df_imdb_title_principals = df_dic["df_imdb_title_principals"]

df_rt_movie_info_tsv = df_dic["df_rt_movie_info_tsv"]
df_rt_reviews_tsv = df_dic["df_rt_reviews_tsv"]

df_tmdb_movies_csv = df_dic["df_tmdb_movies_csv"]

df_tn_movie_budgets_csv = df_dic["df_tn_movie_budgets_csv"]

df_bom_movie_gross_csv = df_dic["df_bom_movie_gross_csv"]


<a id="EDA"></a>

## EXPLORATORY DATA ANALYSIS

Now that the datastes are loaded into DataFrames, we will explore the datasets to understand the data present in each. Later we will drill into the data to understand possible null values and start cleaning the same for joining and use. 

As we see the data for the first time, we will make note of potential areas for cleanup.

In [127]:
df_imdb_title_crew.head(2)

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"


As we see above the all columns seem to contain secondary keys, or it could also contain useless data. We will verify as we see other IMBD datasets, and considering possible joins.

In [130]:
df_imdb_title_akas.head(2)

,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0


Above we see that this df contains movie titles, and lists (in binary) if the title is the original, as well as the region and language it was released in. We will find more about attribute values later on. For now we are assuming original titles are pre-release or location specific. 

We can also see that the "title id" column has a similar format to the "tconst" column in the previous df.

In [131]:
df_imdb_title_ratings.head(2)

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559


It now seems that the "tconst" column in the first df joins with this df rather than the "df_imdb_title_akas" df. And this df provides the ratings as well as number of votes for each score. 

In [207]:
df_imdb_name_basics.head(2)

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"


In this df we see data associated with crew members and their associated movies. Given the nature of our study, we will probably ignore this table altogether, unless it proves necessary for joining tables. 

In [208]:
df_imdb_title_basics.head(2)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


Here we see the genre data and that it can be joined to the "df_imdb_title_ratings" df. 

In [209]:
df_imdb_title_principals.head(2)

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN


This df contains a joining column for all other columns, except the "df_imdb_title_akas" df. It provides parsed information on the roles of the cast and crew related dataframe. 

In [210]:
df_rt_reviews_tsv.head(2)

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"


It seems the above dataframe contains review information, and names and publishing means of the critic (also read "movie-viewer") who posted the same. It could be joined to specific movie using the id column. 

We will verify if this is the case with the following df. 

In [212]:
df_rt_movie_info_tsv.head(2)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One


Our original assumption seems correct in that the RottenTomatoes DataFrames can be joined using the "id" column in the previous table. However, we do not have the name of the film which might be useful for our word count analysis - however, it might not be necessary if we already have enough data samples from IMDB, for example.  

Additionally this dataframe contains "writer", "director", "dvd_date", and "studio" columns which will not be useful for our analysis, but the "director" and "writer" columns could be used for joining to movie titles. We will also need to further investigate the "rating" columns as it seems that the column in this table is specifying the Motion Picture Association (MPA) rating system, rather than point ratings associated with a review.

In [214]:
df_tmdb_movies_csv.head(2)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610


In the above we see that the "Unnamed:0" column can probably be deleted as it seems to contain an index rather than the primary key, as there is another column named "id". 

We can also probably delete one of the title columns. We will be keeping the "title" column, rather than the "original title" column, as we are making the assumption that the latter contains titles used while working on the movie (or prerelease titles).

In [148]:
df_tn_movie_budgets_csv.head(2)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


In [149]:
df_bom_movie_gross_csv.head(2)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


In the above we can see both dataframes contain financial data.

We don't need both dataframes and will choose the former as it contains the full release date and the production budget. Both will be useful for our analysis, and the exact release date can allow us more granularity on when to release Microsoft's first film.  

<a id="Iec"></a>
## Initial exploration and cleanup

Let's take a quick recap on what we've learned from each dataset so far.

1. df_imdb_title_crew: is a joining table, it joins between df_imdb_title_ratings and df_imdb_title_basics, which we can use to title ratings, gender, gross income and ROI (useful for testing our first and second hypotheses) by joining on movie titles with the df_tn_movie_budgets_csv <br><br>

2. df_tn_movie_budgets_csv: contains financial information as well as the name of the movies, which we will use to join on IMDB tables. <br><br>

3. df_tmdb_movies_csv: can join using the original title fields (assuming tmdb is a misnomer for imdb) on df_imdb_title_basics, which can inform our first and third hypotheses (as it could tell us which title is in english for our vectorizer). <br><br>

4. df_rt_movie_info_tsv: can join with df_rt_reviews_tsv using the id columns, and we can then compare ratings, genre and release date, as well as gross domestic income for hypotheses 1,2 and 4.  <br><br>

Now we will start examining and cleaning the data. Let's make note that for our analytical purposes df_imdb_title_principals and df_imdb_name_basics won't be useful, so we will br ignoring these tables altogether.

During the cleanup process we will be looking at potential missing data that could be expressed as other than null. Unique values that could be indicative of missing data, include the following:
- Impossible values
- Null categories
- 0's where not logical


In [ ]:
import 

In [215]:
df_imdb_title_ratings.info() 
# Since there are no noticeable nulls, and we can 
# safely assume all columns are foreign keys given our 
# initial lookthrough, we will continue with the next df. 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
tconst           73856 non-null object
averagerating    73856 non-null float64
numvotes         73856 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [217]:
df_imdb_title_ratings.describe(include=np.object)
# Given our initial lookthrough, and that 
# the "tconst" column contains all unique values
# we can safely assume they are primery and a foreign key.

,tconst
count,73856
unique,73856
top,tt5503266
freq,1


In [219]:
df_imdb_title_ratings.describe() 
# Here we can note that the averages for ratings
# don't vary significantly. 

,averagerating,numvotes
count,73856.000000,7.385600e+04
mean,6.332729,3.523662e+03
std,1.474978,3.029402e+04
min,1.000000,5.000000e+00
25%,5.500000,1.400000e+01
50%,6.500000,4.900000e+01
75%,7.400000,2.820000e+02
max,10.000000,1.841066e+06


In [229]:
for column in df_imdb_title_ratings.columns:
    print(df_imdb_title_ratings[column].value_counts())
# None of the following seems to be possible 
# null values so let's move on.

tt5503266    1
tt1918927    1
tt6398666    1
tt4262042    1
tt2234449    1
            ..
tt4418268    1
tt2112178    1
tt4621790    1
tt2536700    1
tt3010604    1
Name: tconst, Length: 73856, dtype: int64
7.0     2262
6.6     2251
7.2     2249
6.8     2239
6.5     2221
        ... 
9.6       18
10.0      16
9.8       15
9.7       12
9.9        5
Name: averagerating, Length: 91, dtype: int64
6        2875
5        2699
7        2476
8        2167
9        1929
         ... 
18286       1
16289       1
1958        1
4007        1
4098        1
Name: numvotes, Length: 7349, dtype: int64


In [231]:
for column in df_imdb_title_basics.columns:
    print(df_imdb_title_basics[column].value_counts())
    
print(df_imdb_title_basics.info())
df_imdb_title_basics.describe() 

# Again we see no indication of othery types of null values
# and we see little effect on the "genre" and "runtime_min"
# columns

tt8110600    1
tt5097244    1
tt3250890    1
tt4593456    1
tt5916894    1
            ..
tt6922126    1
tt5149530    1
tt1686056    1
tt7528282    1
tt1568930    1
Name: tconst, Length: 146144, dtype: int64
Home                                     24
The Return                               20
Broken                                   20
Homecoming                               16
Alone                                    16
                                         ..
Homeland Nation: Mescalero Apache         1
Dream Journal, May 2016-February 2017     1
Vor Burgdorf danach                       1
The End of Ageing                         1
Second souffle                            1
Name: primary_title, Length: 136071, dtype: int64
Broken                    19
Home                      18
The Return                17
Freedom                   13
Homecoming                13
                          ..
Hey                        1
Chance Pe Dance            1
Eu Sou Carlos Imperial    

,start_year,runtime_minutes
count,146144.000000,114405.000000
mean,2014.621798,86.187247
std,2.733583,166.360590
min,2010.000000,1.000000
25%,2012.000000,70.000000
50%,2015.000000,87.000000
75%,2017.000000,99.000000
max,2115.000000,51420.000000


In [244]:
# Let's continue by removing columns we won't 
# need for our analysis

df_imdb_title_basics = df_imdb_title_basics.drop("runtime_minutes", axis=1).head(2)

In [245]:
# Also, since we will need genre, let's remove 
# columns with nulls in the genre column 
# since these would constitute only  the 3% of the data.
df_imdb_title_basics = df_imdb_title_basics["genres"].dropna()
df_imdb_title_basics.info()


AttributeError: 'Series' object has no attribute 'info'

In [237]:
for column in df_tn_movie_budgets_csv.columns:
    print(df_tn_movie_budgets_csv[column].value_counts())
    
df_tn_movie_budgets_csv.info()

# Again no noticeable null placeholder values and no nulls

4      58
53     58
61     58
65     58
69     58
       ..
85     57
100    57
96     57
92     57
97     57
Name: id, Length: 100, dtype: int64
Dec 31, 2014    24
Dec 31, 2015    23
Dec 31, 2010    15
Dec 31, 2008    14
Dec 31, 2009    13
                ..
Dec 31, 1990     1
Sep 30, 1983     1
Apr 20, 2018     1
Dec 20, 1995     1
May 31, 1996     1
Name: release_date, Length: 2418, dtype: int64
Home                                    3
Halloween                               3
King Kong                               3
Venom                                   2
The Love Letter                         2
                                       ..
Dear Wendy                              1
Freddyâs Dead: The Final Nightmare    1
Nurse Betty                             1
O (Othello)                             1
The Nutcracker                          1
Name: movie, Length: 5698, dtype: int64
$20,000,000    231
$10,000,000    212
$30,000,000    177
$15,000,000    173
$25,000,000    171
 

In [240]:
# Dropping id column which won't be useful for us
df_tn_movie_budgets_csv.drop("id", axis=1).head(2)

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


In [241]:
for column in df_tmdb_movies_csv.columns:
    print(df_tmdb_movies_csv[column].value_counts())
    
df_tmdb_movies_csv.info()


2047     1
5448     1
3371     1
13612    1
15661    1
        ..
12979    1
2740     1
693      1
6838     1
0        1
Name: Unnamed: 0, Length: 26517, dtype: int64
[99]                             3700
[]                               2479
[18]                             2268
[35]                             1660
[27]                             1145
                                 ... 
[16, 35, 99]                        1
[10402, 16, 18]                     1
[12, 18, 14, 9648]                  1
[10402, 10751, 14, 10770, 35]       1
[99, 12, 36]                        1
Name: genre_ids, Length: 2477, dtype: int64
292086    3
463839    3
11976     3
391872    3
416572    3
         ..
356987    1
350846    1
479871    1
500353    1
524288    1
Name: id, Length: 25497, dtype: int64
en    23291
fr      507
es      455
ru      298
ja      265
      ...  
ha        1
hy        1
bo        1
si        1
af        1
Name: original_language, Length: 76, dtype: int64
Eden               